In [1]:
import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#!ls ../input/contrailspackages

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import numpy as np
#pip install /input/contrailsunetresnet26-train/frozen_packages
!pip install --no-index --find-links="../input/contrailspackages" segmentation-models-pytorch
import segmentation_models_pytorch as smp
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.checkpoint as C
import torchvision.transforms.functional as fn
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torchvision import models

#from torchsummary import summary

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device

Looking in links: ../input/contrailspackages
Processing /kaggle/input/contrailspackages/segmentation_models_pytorch-0.3.3-py3-none-any.whl
Processing /kaggle/input/contrailspackages/pretrainedmodels-0.7.4.tar.gz (from segmentation-models-pytorch)
  Preparing metadata (setup.py) ... - \ done
Processing /kaggle/input/contrailspackages/efficientnet_pytorch-0.7.1.tar.gz (from segmentation-models-pytorch)
  Preparing metadata (setup.py) ... - done
Processing /kaggle/input/contrailspackages/munch-4.0.0-py2.py3-none-any.whl (from pretrainedmodels==0.7.4->segmentation-models-pytorch)
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=95554dda98372cd24547443da86b4023499416f1f7b7737516c0d3c8f37eed71
  Stored in directory: /root/.cache/pip/wheels/9e/af/1f/cc11e21b4a5e5f613459a28ceee273fee067da79d75253be92
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=700f517d7c751527c1689

device(type='cuda')

In [2]:
model = torch.load('/kaggle/input/contrailsunetresnet26-train/model_epoch_50_dice_0.3013.pt')
model.eval()

Unet(
  (encoder): ResNestEncoder(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNestBottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [3]:
class ContrailDataset(Dataset):
    
    def __init__(self, base_dir, data_type='test'):
        
        self.base_dir = base_dir
        self.data_type = data_type
        self.record = os.listdir(self.base_dir +'/'+ self.data_type)
       
        #self.resize_image = T.Resize(CFG.resize_value,interpolation=T.InterpolationMode.BILINEAR,antialias=True)
        #self.resize_mask = T.Resize(CFG.resize_value,interpolation=T.InterpolationMode.NEAREST,antialias=True)
   
    def __len__(self):
        return len(self.record)

    def __getitem__(self, idx):
        
        record_id = self.record[idx]
        record_dir = os.path.join(self.base_dir, self.data_type, record_id)
        
        with open(os.path.join(record_dir, 'band_11.npy'), 'rb') as f:
            band11 = np.load(f)
        with open(os.path.join(record_dir, 'band_14.npy'), 'rb') as f:
            band14 = np.load(f)
        with open(os.path.join(record_dir, 'band_15.npy'), 'rb') as f:
            band15 = np.load(f)
            
        _T11_BOUNDS = (243, 303)
        _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
        _TDIFF_BOUNDS = (-4, 2)

        def normalize_range(data, bounds):
            """Maps data to the range [0, 1]."""
            return (data - bounds[0]) / (bounds[1] - bounds[0])

        r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
        g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
        b = normalize_range(band14, _T11_BOUNDS)
        false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
        
        false_color = torch.from_numpy(false_color)
        
        idx = torch.LongTensor([4])
        false_color = false_color.index_select(3, idx)
        false_color = torch.squeeze(false_color)
        false_color = torch.permute(false_color, (2, 1, 0))
        
        return false_color
        

In [4]:
test_data = ContrailDataset(base_dir='/kaggle/input/google-research-identify-contrails-reduce-global-warming', 
                            data_type='test')
test_dataloader = DataLoader(
    test_data, 
    batch_size=1, 
    shuffle=False, 
    num_workers= 4 if torch.cuda.is_available() else 0,
    pin_memory=True,
    drop_last = True
)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s


def rle_decode(mask_rle, shape=(256, 256)):
    '''
    mask_rle: run-length as string formatted (start length)
              empty predictions need to be encoded with '-'
    shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''

    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    if mask_rle != '-': 
        s = mask_rle.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
        starts -= 1
        ends = starts + lengths
        for lo, hi in zip(starts, ends):
            img[lo:hi] = 1
    return img.reshape(shape, order='F')  # Needed to align to RLE direction

In [6]:
from pathlib import Path
data_path = Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming')
import pandas as pd

submission = pd.DataFrame(columns=['record_id', 'encoded_pixels'])
submission = submission.set_index('record_id')


def predict_data(submission):
    record_ids = [record for record in (data_path / 'test').iterdir() if record.is_dir()]
    idx = 0
    with torch.no_grad():
        bar = tqdm(test_dataloader)
        for image in bar:
            image = image.to(device)
            pred = model.predict(image)
            
            pred = pred[0][0]

            pred[pred > 0.5] = 1
            pred[pred <= 0.5] = 0

            pred = pred.cpu()
            submission.loc[int(record_ids[idx].stem)] = list_to_string(rle_encode(pred))
            idx += 1
            
        return submission

submission = predict_data(submission)
submission.to_csv('submission.csv')

  0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
